<a href="https://colab.research.google.com/github/MeikoFudo/IT-school/blob/main/%D0%9F%D1%80_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_1_%D0%98%D0%B3%D0%B8%D1%82%D1%8F%D0%BD_%D0%90%D0%BD%D0%B0%D1%81%D1%82%D0%B0%D1%81%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=bc56fb0464239bc556ba61950a2e019674b33e515386c823786cdc82cd8fe9c2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26114 sha256=0250440c2a41f8393f0521f509cb16779c81ba7fcbc27c166a4757b0aa4f0ede
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b73

Попытка решения данной задачи с помощью выделения коллокаций 


In [ ]:
#Импорт необходимых для задачи библиотек
from natasha import MorphVocab, Doc, Segmenter 
from random import choice, randint
from string import punctuation

#Открываю файл с несколькими текстами на осетинском языке
file = open('oss_text.txt', 'r')
ose_text = file.read()
file.close()

myPunctuation = ['!', '.', '?', '...']

#Токенизация 
morph_vocab = MorphVocab()
segmenter = Segmenter(morph_vocab)
doc = Doc(ose_text)
doc.segment(segmenter)

#Добавляю токены(слова) в список 
words = []
for token in doc.tokens:
    if token.text.isalpha():
        words.append(token.text)

#print(words)

def generate_word(word):
    cnt = words.count(word)
    if cnt != 0:
        ind = randint(1, cnt)
        k = 0
        for i, w in enumerate(words):
            if w == word:
                k += 1
                if k == ind:
                    if ind == len(words) - 1:
                        return choice(words)
                    else:
                        return words[i + 1]
    else:
        return choice(words)


def generate_sentence(begin_word):
    n = randint(5, 10)
    sen = begin_word
    new_word = begin_word
    for i in range(n):
        new_word = generate_word(new_word)
        sen += " " + new_word
    return sen


words = list(filter(lambda x: x not in punctuation, words))

print(generate_sentence("Афтæ")+choice(myPunctuation))


Афтæ цъус куы нæ фæччы хъазын æмæ Ахмæтимæ не вдисынц.


Попытка решения данной задачи с помощью выделения коллокаций 

In [ ]:
#Импорт необходимых для задачи библиотек
from natasha import MorphVocab, Doc, Segmenter
from random import choice, randint
from string import punctuation
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

#Открываю файл с несколькими текстами на осетинском языке
file = open('oss_text.txt', 'r')
ose_text = file.read()
file.close()

#Токенизация 
morph_vocab = MorphVocab()
segmenter = Segmenter(morph_vocab)
doc = Doc(ose_text)
doc.segment(segmenter)

#Добавляю токены(слова) в список 
words = []
for token in doc.tokens:
    if token.text.isalpha():
        words.append(token.text)

#Cоздаю экземпляр класса, который предоставляет методы для подсчета статистических связей между биграммами 
bigram_measures = BigramAssocMeasures() 
#Нахожу коллокации-биграммы в тексте, представленном списком слов
finder = BigramCollocationFinder.from_words(words)
#Фильтрую и оставляю только те, что встречаются не менее 5 раз 
finder.apply_freq_filter(5)
#Добавляю в список 30 коллокаций с самым высоким PMI(Та самая штука из курса ВШЭ по анализу текстов)
collocations = finder.nbest(bigram_measures.pmi, 30)
#print(type(collocations))

import random

def generate_sentence(words, collocations, n):

    """
    На каждом шаге функция generate_sentence выбирает случайное слово из списка слов, 
    а затем выбирает следующее слово из списка коллокаций,
    если предыдущее слово встречалось в первой части коллокации.
    Если таких коллокаций нет, то выбирается новое случайное слово из списка слов.
    """
    sentence = [random.choice(words)]
    while len(sentence) < n: #Генерирую предложение длины n
        last_word = sentence[-1]
        for collocation in collocations:
            if last_word == collocation[0]:
                sentence.append(collocation[1])
                break
        else:
            sentence.append(random.choice(words))
    return ' '.join(sentence)

n = int(input('Сколько ещё бреда тебе надобно старче?'))
print(generate_sentence(words, collocations, n))

Сколько ещё бреда тебе надобно старче?6
хъуамæ сæ къуымы æхсыстой уыдон Принц
